# Intelligent Expense Processing: A Guided Tutorial on Building Agents using AWS Strands with Llama4 Scout and Maverick on Amazon Bedrock 

## Overview
This comprehensive tutorial is designed to guide you through the process of developing and deploying  multi-agent systems using AWS Strands on Amazon Bedrock. Specifically, you will gain hands-on experience in:

1. Creating multiple autonomous agents leveraging the Llama4 Scout and Maverick models, integrated with specialized tools on Amazon Bedrock, to demonstrate the capabilities of AWS Strands.
2. Hosting a fully functional "Expense processing agent" utilizing the Amazon Bedrock AgentCore Runtime, showcasing the seamless integration of AI-driven automation and enterprise workflows.

By the end of this tutorial, you will have acquired the knowledge and skills necessary to design, develop, and deploy sophisticated agent-based applications on Amazon Bedrock with Llama4 models and AWS Strands.

### Tutorial Architecture

<center><img src='images/Agent_workshop.png'></center>

### Pre-requisites

Please ensure that the current IAM role got the following permissions.

    a. AgentCore full access - Please refer to (https://docs.aws.amazon.com/aws-managed-policy/latest/reference/BedrockAgentCoreFullAccess.html )
    b. Amazon Bedrock - Please refer to (https://docs.aws.amazon.com/aws-managed-policy/latest/reference/AmazonBedrockFullAccess.html)
    c. IAM access - Please refer to (https://docs.aws.amazon.com/aws-managed-policy/latest/reference/IAMFullAccess.html)
    d. S3 access - Please refer to (https://docs.aws.amazon.com/aws-managed-policy/latest/reference/AmazonS3FullAccess.html)

### Install Dependencies

In [ ]:
!pip install -r requirements.txt --force --no-cache --quiet

In [ ]:
import sagemaker
role = sagemaker.get_execution_role()

### Function to upload receipt images into S3 bucket

In [ ]:
import boto3
import os

def upload_directory_to_s3(image_dir, bucket_name, s3_prefix):
    """
    Uploads all files from a local directory to an S3 bucket.

    Args:
        local_directory_path (str): The path to the local directory to upload.
        bucket_name (str): The name of the S3 bucket.
        s3_prefix (str, optional): An optional S3 key prefix to add to the uploaded files.
                                   This helps in organizing files within the S3 bucket.
    """
    s3_client = boto3.client('s3')

    if not os.path.isdir(image_dir):
        print(f"Error: Local directory '{image_dir}' not found.")
        return
    
    entries = os.listdir(image_dir)
    images = [image_dir+"/"+entry for entry in entries if os.path.isfile(os.path.join(image_dir, entry))]
    
    for img in images:
        try:
            s3_key = f"{s3_prefix}/{img.split('/')[-1]}"
            s3_client.upload_file(img, bucket_name, s3_key)
            print(f"Uploaded '{img}' to 's3://{bucket_name}/{s3_prefix}'")
        except Exception as e:
            print(f"Error uploading '{img}': {e}")



### Define S3 Bucket and Prefix where the images will be uploaded

In [ ]:
# Replace with your actual directory path and S3 bucket details
images_dir = "./Receipt_images" #Local dir path where the receipt images are available

s3_bucket = '' #update your custom s3 bucket name here if needed
if not s3_bucket: #If there is no custom bucket defined, consider Sagemaker's default bucket
    import sagemaker
    session = sagemaker.Session()
    s3_bucket = session.default_bucket()    
    print(s3_bucket)

s3_prefix = "doc_processing/images" #Upload into a "directory" within the bucket



In [ ]:
#Upload all the images from local dir to S3
upload_directory_to_s3(images_dir, s3_bucket, s3_prefix)

### Define employee related variables

In [ ]:
import json
inputs = {"emp_first_name": "Jane",
           "emp_last_name": "Doe",
           "emp_num": 12345,
           "cost_center": 98763,
           "division": "Sales",
           "bucket": s3_bucket,
           "images_prefix": s3_prefix
        }

inputs = json.dumps(inputs)

#### Invoking local agent

In [ ]:
from src.main_local import run_expense_processor
expense_report = run_expense_processor(inputs)

In [ ]:
#Function to read the output using pandas
import pandas as pd

def df_out(expense_report):
    df = pd.json_normalize(expense_report)
    return df

In [ ]:
exp_report_sum = df_out(expense_report)
exp_report_sum

In [ ]:
#Exceptions details from DF
exp_report_sum['EXCEPTIONS_SUMMARY'][0]

In [ ]:
#Expenses details from DF 
exp_report_sum['Expenses'][0]

### Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to add the following in out orchestrator agent ```src/main_AgentCore.py```:

Please check ```src/main_local.py```and  ```src/main_AgentCore.py``` on modifications done.

1. Import the Runtime App with ```from bedrock_agentcore.runtime import BedrockAgentCoreApp```
2. Initialize the App in our code with ```app = BedrockAgentCoreApp()```
3. Decorate the invocation function with the ```@app.entrypoint decorator```

Let AgentCoreRuntime control the running of the agent with ```app.run()```

#### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [ ]:
#Local Version before making changes
!cat src/main_local.py 

In [ ]:
!diff src/main_local.py src/main_AgentCore.py

In [ ]:
#After changes specific to AgentCore Runtime
!cat src/main_AgentCore.py 

#### Deploying the agent to AgentCore Runtime
The CreateAgentRuntime operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent.

Note: Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

#### Configure AgentCore Runtime deployment
First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<center><img src='images/configure.png'></center>

#### IAM service role
To run an AgentCore Runtime you must create a service role and add all neccessary permissions. The service role allows AgentCore Runtime to perform actions on your behalf in your AWS account. Please refer to [Documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/automatic-service-roles.html.) for more details.

In [ ]:
#Create IAM role
iam = boto3.client('iam')
aws_acct = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name



In [ ]:
AC_policy_doc= json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AssumeRolePolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": "{}".format(aws_acct)
                },
                "ArnLike": {
                    "aws:SourceArn": "arn:aws:bedrock-agentcore:{}:{}:*".format(region, aws_acct)
                }
            }
        }
    ]
}
)

In [ ]:
import datetime

role_name="AgentCoreRuntimeWorkshop-{}".format(str(datetime.datetime.now().timestamp()).split('.')[0])
create_role_response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument = AC_policy_doc
)

In [ ]:
role_arn = create_role_response["Role"]["Arn"]

role_arn

In [ ]:
AC_permission_policy = json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer"
            ],
            "Resource": [
                "arn:aws:ecr:{}:{}:repository/*".format(region, aws_acct)
            ]        
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:CreateLogGroup"
            ],
            "Resource": [
                "arn:aws:logs:{}:{}:log-group:/aws/bedrock-agentcore/runtimes/*".format(region, aws_acct)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogGroups"
            ],
            "Resource": [
                "arn:aws:logs:{}:{}:log-group:*".format(region, aws_acct)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:{}:{}:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*".format(region, aws_acct)
            ]
        },
        {
            "Sid": "ECRTokenAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken"
            ],
            "Resource": "*"
        },
        {
        "Effect": "Allow", 
        "Action": [ 
            "xray:PutTraceSegments", 
            "xray:PutTelemetryRecords", 
            "xray:GetSamplingRules", 
            "xray:GetSamplingTargets"
            ],
         "Resource": [ "*" ] 
         },
         {
            "Effect": "Allow",
            "Resource": "*",
            "Action": "cloudwatch:PutMetricData",
            "Condition": {
                "StringEquals": {
                    "cloudwatch:namespace": "bedrock-agentcore"
                }
            }
        },
        {
            "Sid": "GetAgentAccessToken",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessToken",
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
            ],
            "Resource": [
              "arn:aws:bedrock-agentcore:{}:{}:workload-identity-directory/default".format(region, aws_acct),
              "arn:aws:bedrock-agentcore:{}:{}:workload-identity-directory/default/workload-identity/agentName-*".format(region, aws_acct)
            ]
        },
         {
			"Sid": "BedrockModelInvocation",
			"Effect": "Allow",
			"Action": [
				"bedrock:InvokeModel",
				"bedrock:InvokeModelWithResponseStream",
				"bedrock:ApplyGuardrail"
			],
			"Resource": [
				"arn:aws:bedrock:*::foundation-model/*",
				"arn:aws:bedrock:{}:{}:*".format(region, aws_acct)
			]
		},
        {
			"Sid": "ListObjectsInBucket",
			"Effect": "Allow",
			"Action": [
				"s3:ListBucket"
			],
			"Resource": [
				"arn:aws:s3:::{}".format(s3_bucket)
			]
		},
		{
			"Sid": "AllObjectActions",
			"Effect": "Allow",
			"Action": "s3:*Object",
			"Resource": [
				"arn:aws:s3:::{}/*".format(s3_bucket)
			]
		}
    ]
}

                          )


In [ ]:
iam_ac_response = iam.put_role_policy(
    RoleName=role_name,
    PolicyName="agentcore-perm",
    PolicyDocument=AC_permission_policy
)
iam_ac_response

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import datetime
import random

boto_session = Session()
region = boto_session.region_name

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime into a string (e.g., 'YYYYMMDD_HHMMSS')
datetime_string = now.strftime("%Y%m%d%H%M%S")

# Generate a random integer (e.g., between 1000 and 9999)
random_number = random.randint(1000, 9999)

# Combine the datetime string and the random number into a single string
random_string = f"{datetime_string}_{random_number}"

agentcore_runtime = Runtime()
agent_name = f"expense_claim_processor_{random_string}"
response = agentcore_runtime.configure(
    entrypoint="src/main_AgentCore.py",
    execution_role=role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)


#### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

<center><img src='images/launch_1.png'></center>

#### Write a custom Dockerfile

In [ ]:
%%writefile Dockerfile
FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim
WORKDIR /app

#COPY all dependencies
COPY src/main_AgentCore.py /app/main_AgentCore.py
COPY src/ocr_agent.py /app/ocr_agent.py
COPY src/policy_checker.py /app/policy_checker.py
COPY src/cur_convert.py /app/cur_convert.py
COPY src/structured_out.py /app/structured_out.py
COPY src/travel_policy.txt /app/travel_policy.txt

# Configure UV for container environment
ENV UV_SYSTEM_PYTHON=1 UV_COMPILE_BYTECODE=1

COPY requirements.txt requirements.txt
# Install from requirements file
RUN uv pip install -r requirements.txt
RUN uv pip install aws-opentelemetry-distro>=0.10.1

# Set AWS region environment variable
ENV AWS_REGION=us-east-1
ENV AWS_DEFAULT_REGION=us-east-1

# Signal that this is running in Docker for host binding logic
ENV DOCKER_CONTAINER=1

# Create non-root user
RUN useradd -m -u 1000 bedrock_agentcore
USER bedrock_agentcore

EXPOSE 8080
EXPOSE 8000

# Copy entire project (respecting .dockerignore)
COPY . .

# Use the full module path
CMD ["opentelemetry-instrument", "python", "-m", "main_AgentCore"]

In [ ]:
launch_result = agentcore_runtime.launch()

#### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime
<center><img src='images/launch_2.png'></center>

In [ ]:
launch_result = agentcore_runtime.launch()

#### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

#### Invoking AgentCore Runtime
Finally, we can invoke our AgentCore Runtime with a payload
<center><img src='images/invoke.png'></center>

In [ ]:
import json
payload = {"emp_first_name": "Jane",
           "emp_last_name": "Doe",
           "emp_num": 12345,
           "cost_center": 98763,
           "division": "Sales",
           "bucket": s3_bucket,
           "images_prefix": s3_prefix
        }

payload = json.dumps(payload)

In [ ]:
invoke_response = agentcore_runtime.invoke(payload)


#### Processing invocation results
We can now process our invocation results to include it in an application

In [ ]:
response_json = json.loads("".join(invoke_response['response']))
df_result = df_out(response_json)
df_result

#Note: You can also explore the expenses within results as df_result['Expenses'] and df_result['EXCEPTIONS_SUMMARY']

#### Invoking AgentCore Runtime with boto3
Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 ```invoke_agent_runtime``` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=payload
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]


In [ ]:
boto3_response_json = json.loads("".join([event.decode("utf-8") for event in events]))
df_result = df_out(boto3_response_json)
df_result

#Note: You can also explore the expenses within results as df_result['Expenses'] and df_result['EXCEPTIONS_SUMMARY']

### Cleanup (Optional)
Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)